In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split
import torch.optim as optim
import pandas as pd
import numpy as np
import os
from collections import defaultdict, Counter
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold
import optuna
from torchmetrics.classification import F1Score
import pickle
import sys
import warnings
warnings.filterwarnings("ignore")
import random
from sklearn.model_selection import ShuffleSplit
from sklearn.cluster import KMeans,DBSCAN,Birch
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score, v_measure_score

sys.path.insert(1, '/home/wangc90/Data_integration/MOCSS/mocss/code/')
from critic import LinearCritic
# from Supcon import SupConLoss
from contrastive_loss import InstanceLoss, ClusterLoss
import evaluation
from sklearn import metrics
from Data_prep import DataSet_Prep, DataSet_construction
random.seed(2023)
torch.manual_seed(2023)


In [2]:
### variationalAE with concatenated inputs (CNC-VAE)
### In order to train the variational autoencoder, we only need to add the auxillary loss in our training algorithm

class CNC_Encoder(nn.Module):

    """
        concatenate the s1, s2 directly for a joint embedding
    """
    
    def __init__(self, trial):
        
        possible_nodes = [32, 64, 128, 256, 512, 1024]
        possible_dropout = [0, 0.1, 0.2, 0.4, 0.6]

        ### input dimension for omic 1, omic 2 and omic 3
        self.s1_input_dim = 20531
        self.s2_input_dim = 1046
        
        ### layer 1 output dimension for omic 12
        self.l1_s12_out_dim = trial.suggest_categorical("l1_s12_out_dim", possible_nodes)
        
        ### layer 2 output dimension for omic 12
        self.l2_s12_out_dim = trial.suggest_categorical("l2_s12_out_dim", possible_nodes)
        
        ### layer 3 output dimension for omic 12
        self.l3_s12_out_dim = trial.suggest_categorical("l3_s12_out_dim", possible_nodes)
        
        
        ### output dimension for common embedding dimension
        self.common_embed_dim = trial.suggest_categorical("common_embed_dim", possible_nodes)
        
        
        super(CNC_Encoder, self).__init__()
        
        ### encoder structure:
        ### first layer
        self.l1_s12 = nn.Linear(self.s1_input_dim + self.s2_input_dim,
                                 self.l1_s12_out_dim)
        ### corresponding bn layer
        self.l1_s12_bn = nn.BatchNorm1d(self.l1_s12_out_dim)
        ### corresponding dropout layer
        l1_s12_drop_rate = trial.suggest_categorical("l1_s12_drop_rate", possible_dropout)
        self.drop_l1_s12 = nn.Dropout(p=l1_s12_drop_rate)
        
        
        ### second layer
        self.l2_s12 = nn.Linear(self.l1_s12_out_dim,  self.l2_s12_out_dim)
        ### corresponding bn layer
        self.l2_s12_bn = nn.BatchNorm1d(self.l2_s12_out_dim)
        ### corresponding dropout layer
        l2_s12_drop_rate = trial.suggest_categorical("l2_s12_drop_rate", possible_dropout)
        self.drop_l2_s12 = nn.Dropout(p=l2_s12_drop_rate)
        
        
        ### third layer
        self.l3_s12 = nn.Linear(self.l2_s12_out_dim,  self.l3_s12_out_dim)
        ### corresponding bn layer
        self.l3_s12_bn = nn.BatchNorm1d(self.l3_s12_out_dim)
        ### corresponding dropout layer
        l3_s12_drop_rate = trial.suggest_categorical("l3_s12_drop_rate", possible_dropout)
        self.drop_l3_s12 = nn.Dropout(p=l3_s12_drop_rate)
        
        
        ### embedding layers 
        self.embed_s12 = nn.Linear(self.l3_s12_out_dim, self.common_embed_dim)
        self.embed_s12_bn = nn.BatchNorm1d(self.common_embed_dim)
        ### corresponding dropout layer
        embed_s12_drop_rate = trial.suggest_categorical("embed_s12_drop_rate", possible_dropout)
        self.drop_embed_s12 = nn.Dropout(p=embed_s12_drop_rate)
        
        

    def forward(self, s1, s2, labels=None):
        
        s12 = torch.cat((s1, s2), dim=1)
        s12_ = self.drop_l1_s12(self.l1_s12_bn(F.relu(self.l1_s12(s12))))
        s12_ = self.drop_l2_s12(self.l2_s12_bn(F.relu(self.l2_s12(s12_))))
        s12_ = self.drop_l3_s12(self.l3_s12_bn(F.relu(self.l3_s12(s12_))))
        z12 = self.drop_embed_s12(self.embed_s12_bn(F.relu(self.embed_s12(s12_))))
        
        return z12, labels
    

class CNC_Decoder(nn.Module):
    
    def __init__(self, trial):
        
        possible_nodes = [32, 64, 128, 256, 512, 1024]
        possible_dropout = [0, 0.1, 0.2, 0.4, 0.6]
        
        self.s1_input_dim = CNC_Encoder(trial).s1_input_dim
        self.s2_input_dim = CNC_Encoder(trial).s2_input_dim
        
        self.common_embed_dim = CNC_Encoder(trial).common_embed_dim
        
        self._embed_s1_out_dim = trial.suggest_categorical("_embed_s1_out_dim", possible_nodes)
        self._l3_s1_out_dim = trial.suggest_categorical("_l3_s1_out_dim", possible_nodes)
        self._l2_s1_out_dim = trial.suggest_categorical("_l2_s1_out_dim", possible_nodes)
        self._l1_s1_out_dim = self.s1_input_dim
        
        
        self._embed_s2_out_dim = trial.suggest_categorical("_embed_s2_out_dim", possible_nodes)
        self._l3_s2_out_dim = trial.suggest_categorical("_l3_s2_out_dim", possible_nodes)
        self._l2_s2_out_dim = trial.suggest_categorical("_l2_s2_out_dim", possible_nodes)
        self._l1_s2_out_dim = self.s2_input_dim
        
        
        
        super(CNC_Decoder, self).__init__()
        
        #############################################################################
        self._embed_s1 = nn.Linear(self.common_embed_dim, self._embed_s1_out_dim)
        self._embed_s1_bn = nn.BatchNorm1d(self._embed_s1_out_dim)
        _embed_s1_drop_rate = trial.suggest_categorical("_embed_s1_drop_rate", possible_dropout)
        self._drop_embed_s1 = nn.Dropout(p=_embed_s1_drop_rate)
        
        
        self._l3_s1 = nn.Linear(self._embed_s1_out_dim, self._l3_s1_out_dim)
        self._l3_s1_bn = nn.BatchNorm1d(self._l3_s1_out_dim)
        _l3_s1_drop_rate = trial.suggest_categorical("_l3_s1_drop_rate", possible_dropout)
        self._drop_l3_s1 = nn.Dropout(p=_l3_s1_drop_rate)
        
        
        self._l2_s1 = nn.Linear(self._l3_s1_out_dim, self._l2_s1_out_dim)
        self._l2_s1_bn = nn.BatchNorm1d(self._l2_s1_out_dim)
        _l2_s1_drop_rate = trial.suggest_categorical("_l2_s1_drop_rate", possible_dropout)
        self._drop_l2_s1 = nn.Dropout(p=_l2_s1_drop_rate)
        
        
        self._l1_s1 = nn.Linear(self._l2_s1_out_dim, self._l1_s1_out_dim)
        self._l1_s1_bn = nn.BatchNorm1d(self._l1_s1_out_dim)
        _l1_s1_drop_rate = trial.suggest_categorical("_l1_s1_drop_rate", possible_dropout)
        self._drop_l1_s1 = nn.Dropout(p=_l1_s1_drop_rate)
        
        
        
        #############################################################################

        self._embed_s2 = nn.Linear(self.common_embed_dim, self._embed_s2_out_dim)
        self._embed_s2_bn = nn.BatchNorm1d(self._embed_s2_out_dim)
        _embed_s2_drop_rate = trial.suggest_categorical("_embed_s2_drop_rate", possible_dropout)
        self._drop_embed_s2 = nn.Dropout(p=_embed_s2_drop_rate)
        
        
        self._l3_s2 = nn.Linear(self._embed_s2_out_dim, self._l3_s2_out_dim)
        self._l3_s2_bn = nn.BatchNorm1d(self._l3_s2_out_dim)
        _l3_s2_drop_rate = trial.suggest_categorical("_l3_s2_drop_rate", possible_dropout)
        self._drop_l3_s2 = nn.Dropout(p=_l3_s2_drop_rate)
        
        
        self._l2_s2 = nn.Linear(self._l3_s2_out_dim, self._l2_s2_out_dim)
        self._l2_s2_bn = nn.BatchNorm1d(self._l2_s2_out_dim)
        _l2_s2_drop_rate = trial.suggest_categorical("_l2_s2_drop_rate", possible_dropout)
        self._drop_l2_s2 = nn.Dropout(p=_l2_s2_drop_rate)
        
        
        self._l1_s2 = nn.Linear(self._l2_s2_out_dim, self._l1_s2_out_dim)
        self._l1_s2_bn = nn.BatchNorm1d(self._l1_s2_out_dim)
        _l1_s2_drop_rate = trial.suggest_categorical("_l1_s2_drop_rate", possible_dropout)
        self._drop_l1_s2 = nn.Dropout(p=_l1_s2_drop_rate)
        
        

    def forward(self, z12):
        
        s1_ = self._drop_embed_s1(self._embed_s1_bn(F.relu(self._embed_s1(z12))))
        s1_ = self._drop_l3_s1(self._l3_s1_bn(F.relu(self._l3_s1(s1_))))
        s1_ = self._drop_l2_s1(self._l2_s1_bn(F.relu(self._l2_s1(s1_))))
        s1_ = self._drop_l1_s1(self._l1_s1_bn(F.relu(self._l1_s1(s1_))))
        
        s1_out = torch.sigmoid(s1_)
        
        
        s2_ = self._drop_embed_s2(self._embed_s2_bn(F.relu(self._embed_s2(z12))))
        s2_ = self._drop_l3_s2(self._l3_s2_bn(F.relu(self._l3_s2(s2_))))
        s2_ = self._drop_l2_s2(self._l2_s2_bn(F.relu(self._l2_s2(s2_))))
        s2_ = self._drop_l1_s2(self._l1_s2_bn(F.relu(self._l1_s2(s2_))))
        
        s2_out = torch.sigmoid(s2_)
        
        
        return s1_out, s2_out
    
    
class CNC_AE(nn.Module):
    def __init__(self, trial):
        
        super(CNC_AE, self).__init__()
        
        self.encoder = CNC_Encoder(trial)
        
        self.decoder = CNC_Decoder(trial)

    def forward(self, s1, s2, labels):
        ### encoder ouput for embeddings
        z12, labels = self.encoder(s1, s2, labels)
        ### decoder output for reconstructed input
        s1_out, s2_out = self.decoder(z12)

        return z12, s1_out, s2_out, labels

In [3]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

In [4]:
def CustomLoss(s1, s2, s1_out, s2_out,
                z12, labels, z1=None, z2=None):
    #         self.alpha = alpha

    ### subtract the mean 

#     s1_out = s1_out - s1_out.mean()
#     s2_out = s2_out - s2_out.mean()
    
    ### normalize the feature vector with length 1
    s1_out = F.normalize(s1_out, p=2, dim=1)
    s2_out = F.normalize(s2_out, p=2, dim=1)
    
    
#     s1 = s1 - s1.mean()
#     s2 = s2 - s2.mean()
    s1 = F.normalize(s1, p=2, dim=1)
    s2 = F.normalize(s2, p=2, dim=1)

    recon_loss = torch.linalg.matrix_norm(s1_out-s1) + torch.linalg.matrix_norm(s2_out-s2)

    return recon_loss

In [5]:
### create a training function that is seamless for all different models
def Objective(device, trial, fold, model, optimizer,
              epochs, train_loader, val_loader):

    
    #### model training on the training set
    
    for epoch in range(epochs):
        
        model.train()
        # record the training loss
        total_recon_loss = 0.0
        total_train = 0.0
        
        for iteration_index, train_batch in enumerate(train_loader):
            view1_train_data, view2_train_data, train_labels = train_batch

            view1_train_data = view1_train_data.type(torch.float32).to(device)
            view2_train_data = view2_train_data.type(torch.float32).to(device)
            train_labels = train_labels.type(torch.LongTensor).to(device)
            
            z12, s1_out, s2_out, labels = \
                model(view1_train_data, view2_train_data, train_labels)
            
#             print(z123.shape, s1_out.shape, s2_out.shape, s3_out.shape)
            train_size = z12.size()[0]
    
            recon_loss = CustomLoss(s1=view1_train_data,\
                                             s2=view2_train_data,\
                                             s1_out=s1_out,\
                                             s2_out=s2_out,\
                                             z12=z12,\
                                             labels=train_labels)
            
#             print(clustering_loss)
            
            loss = recon_loss
            # backward pass
            optimizer.zero_grad()  # empty the gradient from last round

            # calculate the gradient
            loss.backward()
            
#             print(model.weight.grad)
            # update the parameters
            optimizer.step()
            
            total_train += train_size
#             total_clustering_loss += (clustering_loss.item()*train_size)
            total_recon_loss += recon_loss.item()
        
        
        if (epoch + 1) % 10 == 0: 
            print(f'fold {fold+1} epoch {epoch+1}')
            print(f'average train recon loss is: {total_recon_loss/total_train}')
            
            
    #### model evaluation on the validation set at the last epoch and return val_loss
    
    model.eval()
    with torch.no_grad():

        total_recon_loss = 0.0
        total_val = 0.0

        ### collect the embeddings of all the validation set for subsequent clustering measurement
        for iteration_index, val_batch in enumerate(val_loader):
            view1_val_data, view2_val_data, val_labels = val_batch

            view1_val_data = view1_val_data.type(torch.float32).to(device)
            view2_val_data = view2_val_data.type(torch.float32).to(device)
            val_labels = val_labels.type(torch.LongTensor).to(device)

            z12_val, s1_out_val, s2_out_val, labels_val = \
                model(view1_val_data, view2_val_data, val_labels)

            val_size = z12.size()[0]

            recon_loss = CustomLoss(s1=view1_val_data,\
                                         s2=view2_val_data,\
                                         s1_out=s1_out_val,\
                                         s2_out=s2_out_val,\
                                         z12=z12_val,\
                                         labels=labels_val)

            total_val += val_size
            total_recon_loss += recon_loss.item()
        avg_recon_loss = total_recon_loss / total_val
    return avg_recon_loss



class Objective_CV:

    def __init__(self, cv, model, dataset, val_loss_folder):

        self.cv = cv  ## number of CV
        self.model = model  ## pass the corresponding model
        self.dataset = dataset  ## the corresponding dataset object
        self.val_loss_folder = val_loss_folder  ## folder to store the cross_validation accuracy

    def __call__(self, trial):

        ### just use the sequence feature for now
        device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')

        lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
#         lr = 0.001
        l2_lambda = trial.suggest_float("l2_lambda", 1e-8, 1e-5, log=True)
#         l2_lambda = 0
        ### fix and use the maximal allowed batch size
        batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512])
#         batch_size = 100

        ### optimize epoch number
        epochs = trial.suggest_categorical("epoch", [30, 60, 90, 120, 150])
        
        ## choose the optimizer
#         optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop","Adagrad"])
        optimizer_name = 'Adam'

        kfold = KFold(n_splits=self.cv, shuffle=True)

        setup_seed(21)
        
        val_fold_loss = []
                    
        for fold, (train_index, val_index) in enumerate(kfold.split(np.arange(len(self.dataset)))):

            ### get the train and val loader
            train_subset = torch.utils.data.Subset(self.dataset, train_index)
            train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
            

            val_subset = torch.utils.data.Subset(self.dataset, val_index)
            val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=True)
            

            ## model should be initilized here for each fold to have a new model with same hyperparameters

            ### for the model the process the concatenated upper and lower the is_rcm is always False
            Model = self.model(trial).to(device=device)
#             print(Model)

            optimizer = getattr(optim, optimizer_name)(Model.parameters(), lr=lr, weight_decay=l2_lambda)

            val_loss = Objective(device, trial, fold=fold, model=Model, optimizer=optimizer,
                                 epochs=epochs, train_loader=train_loader, val_loader=val_loader)

            val_fold_loss.append(val_loss)

        avg_val_loss = np.mean(val_fold_loss)

        val_loss_path = f"{self.val_loss_folder}/val_loss.csv"

        val_loss_str = '\t'.join([str(i) for i in val_fold_loss])
        with open(val_loss_path, 'a') as f:
            f.write('trial' + str(trial.number) + '\t' + val_loss_str + '\n')

        return avg_val_loss


In [6]:
def CNC_AE_model_selection(num_trial):

    ### where to save the 3-fold CV validation acc

    val_loss_folder = '/home/wangc90/Data_integration/TCGA_model_outputs/model_selection_outputs/CNC_AE/val_loss'
    
    ### wehre to save the detailed optuna results
    optuna_folder = '/home/wangc90/Data_integration/TCGA_model_outputs/model_selection_outputs/CNC_AE/optuna'
    
    
    combined_exp_df = pd.read_csv('/home/wangc90/Data_integration/TCGA_data/TCGA_primary_tumor_data/combined_exp_df.csv', sep='\t')
    combined_miRNA_df = pd.read_csv('/home/wangc90/Data_integration/TCGA_data/TCGA_primary_tumor_data/combined_miRNA_df.csv', sep='\t')

    labels = pd.read_csv('/home/wangc90/Data_integration/TCGA_data/TCGA_primary_tumor_data/labels.csv', sep='\t')['0']

    dataset_prep = DataSet_Prep(data1=combined_exp_df, data2=combined_miRNA_df, label=labels, training_prop=0.8)

    train_key, test_key = dataset_prep.get_train_test_keys()

    feature1_tensors, feature2_tensors, label_tensors = dataset_prep.to_tensor(train_key)
    
    train_dataset = DataSet_construction(feature1_tensors, feature2_tensors, label_tensors)

    print(len(train_dataset))
    
    
    study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=2),
                                direction='minimize')
    
    
    study.optimize(Objective_CV(cv=5, model= CNC_AE,
                                dataset=train_dataset,
                                val_loss_folder=val_loss_folder),
                   n_trials=num_trial, gc_after_trial=True)


    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    with open(optuna_folder+'/optuna.txt', 'a') as f:
        f.write("Study statistics: \n")
        f.write(f"Number of finished trials: {len(study.trials)}\n")
        f.write(f"Number of pruned trials: {len(pruned_trials)}\n")
        f.write(f"Number of complete trials: {len(complete_trials)}\n")

        f.write("Best trial:\n")
        trial = study.best_trial
        f.write(f"Value: {trial.value}\n")
        f.write("Params:\n")
        for key, value in trial.params.items():
            f.write(f"{key}:{value}\n")

    df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete','duration','number'], axis=1)
    df.to_csv(optuna_folder + '/optuna.csv', sep='\t', index=None)

In [ ]:
import time

# Record the start time
start_time = time.time()


CNC_AE_model_selection(100)

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time/60} minutes")

[I 2024-02-23 11:19:51,221] A new study created in memory with name: no-name-395a430c-3bf7-432b-a07e-99d6bb40de14


feature1 and feature2 are being scaled with MinMaxScaler
1494
fold 1 epoch 10
average train recon loss is: 0.11560281889209188
